# The mosquito question

While hiking through the woods of Ontario several years ago getting absolutely swarmed by mosquitoes. As I was being eaten alive, I began musing out loud: is my life in danger?

I've become a little obsessed with finding the answer this question. 

A quick warning: we're talking about blood and death here, so if you're squeamish at all, you may want to mute. Take care of yourself.

Other people out there have tried to answer this:
* [Outdoors.org](https://www.outdoors.org/resources/amc-outdoors/health-and-safety/how-many-mosquito-bites-would-kill-you/)
* [Business Insider](https://www.businessinsider.com/how-many-mosquitoes-drain-person-2016-2)

But I find the detail lacking, so we're going to approach this mathmatically using Pandas and a number of assumptions. Of course, there are so many ways to die from mosquito bites: disease, anaphylaxis etc. But here's what I want to answer today: How many mosquitoes would it take, constantly hovering around you and biting as much as they can, to overcome your body's natural regeneration and threaten your life?

First, let's import pandas.

In [ ]:
import pandas as pd

## The human variables

Let's start with the human variables. Human blood is about 55% plasma, and 45% red blood cells (also some other things, like white blood cells). Because red cells regenerate at a different rate than your plasma, we calculate them both separately.

According to the [American Red Cross website](https://www.redcrossblood.org/faq.html), it takes 4-6 weeks to regenerate the lost red blood cells from a donated pint of blood, and a day to regenerate the lost plasma.

Note that while other attempts to answer this question deal with the amount of blood a human being has in its body, it's irrelevant here, as we're only interested in the tipping point where mosquitoes overwhelm your body's natural regeneration.

In [ ]:
percent_blood_plasma = 0.55 ## Your blood is roughly 55% plasma. https://www.medicalnewstoday.com/articles/321122#shock-and-blood-loss
percent_blood_redcells = 0.45 ## The non-plasma part of blood is made up of mostly red blood cells. https://www.medicalnewstoday.com/articles/321122#shock-and-blood-loss

ml_per_pint = 473

redcell_daily_regen_fastest = ((1 * percent_blood_redcells) / 28) * ml_per_pint # Pints of pure RBC per day. We multiply by percent_blood_redcells to get the regen for a full pint of pure RBC.
redcell_daily_regen_slowest = ((1 * percent_blood_redcells) / 42) * ml_per_pint # This is the slowest that RBCs regenerate.
plasma_daily_regen = ((1 * percent_blood_plasma) / 1) * ml_per_pint  # Pints of pure plasma per day

## The mosquito variables

Now, the mosquito variables. A mosquito can take [5 microlitres of blood (0.005 mL) per meal](https://www.businessinsider.com/how-many-mosquitoes-drain-person-2016-2), and the best estimate I could find on how often a single mosquito can bite you is every 3 days [here](https://mosquitoenemy.com/how-much-blood-can-a-mosquito-drink/).

When mosquitoes bite, they take a volume of blood that is made up of plasma and red blood cells (among other things), in a proportion that is roughly equal to the proportion of each in each human's blood (55% plasma, 45% red blood cells).

In [ ]:
daily_single_mosquito_drain = -0.005 / 3 # This is divided by three because a mosquito can take about 0.005 of blood every three days, at which point the digestion process is complete.

# RBC and plasma will be taken at slightly different rates in every bite.
single_mosquito_drain_rate_plasma = daily_single_mosquito_drain * percent_blood_plasma
single_mosquito_drain_rate_redcells = daily_single_mosquito_drain * percent_blood_redcells

## Building the swarm

Let's build out our swarm of mosquitoes: a table where each row represents the net blood drain of a certain number of mosquitoes.

In [ ]:
## An empty Pandas dataframe.
df = pd.DataFrame()

# A column representing a number of mosquitoes.
df["mosquitoes"] = range(1, 300000)

df["Net plasma regeneration"] = plasma_daily_regen + df['mosquitoes'] * single_mosquito_drain_rate_plasma
df["Net RBC regeneration (slowest)"] = redcell_daily_regen_slowest + df['mosquitoes'] * single_mosquito_drain_rate_redcells
df["Net RBC regeneration (fastest)"] = redcell_daily_regen_fastest + df['mosquitoes'] * single_mosquito_drain_rate_redcells 

df = df.set_index("mosquitoes")

display(df)

Finally, we find the point in each column where the values flip from positive regeneration rate per day to a negative rate per day (the "danger zone").

In [214]:
danger_point_redcells_fastest = df.index[df['Net RBC regeneration (fastest)'] <= 0][0]
danger_point_redcells_slowest = df.index[df['Net RBC regeneration (slowest)'] <= 0][0]
danger_point_plasma = df.index[df['Net plasma regeneration'] <= 0][0]

print("Red blood cell danger zone: " + danger_point_redcells_slowest.astype(str) + "-" + danger_point_redcells_fastest.astype(str) + " mosquitoes.")
print("Blood plasma danger point: " + danger_point_plasma.astype(str) + " mosquitoes.")

Red blood cell danger zone: 6758-10136 mosquitoes.
Blood plasma danger point: 283800 mosquitoes.
